In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2

In [2]:
from pathlib import Path
# Assuming 'config' and 'load_data' class are defined and imported elsewhere

# --- Parameters derived from your framework ---
v: float = 0.5
space: List[str] = ['2', '2']
days: List[str] = ['0', '31']
mm_cond_number: int = 20
# --- End of framework parameters ---

lat_lon_resolution = [int(s) for s in space]
days_s_e = [int(d) for d in days]
days_list = list(range(days_s_e[0], days_s_e[1]))

# These values were not in the framework, so they remain as set in your snippet
years = ['2024']
month_range = [7] 

# Assuming 'config' is available in your environment
output_path = input_path = Path(config.mac_estimates_day_path)

## load ozone data from amarel
data_load_instance = load_data2(config.mac_data_load_path)

# Call the function using the variables from the framework
df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      # <-- Add this
lon_range=[123.0, 133.0]   # <-- Add this
)

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


In [3]:
df_day_aggregated_list = []
df_day_map_list = []
for i in range(31):
    idx_for_datamap = [i*8, (i+1)*8]

    cur_map, cur_df = analysis_map_no_mm, agg_data_no_mm = data_load_instance.load_working_data_keep_ori(
    df_map, 
    idx_for_datamap, 
    ord_mm=None,  # or just omit it
    dtype=torch.float # or just omit it
)
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(df_day_aggregated_list[0].shape)
print(df_day_aggregated_list[1][:10])



torch.Size([36480, 6])
tensor([[  4.9760, 132.9840, 267.0307,  45.0000,   4.9764, 132.9858],
        [  4.9760, 132.8580, 258.8635,  45.0000,   4.9766, 132.8600],
        [  4.9760, 132.7320, 272.0078,  45.0000,   4.9769, 132.7346],
        [  4.9760, 132.6060, 267.8075,  45.0000,   4.9772, 132.6088],
        [  4.9760, 132.4800, 266.1405,  45.0000,   4.9776, 132.4833],
        [  4.9760, 132.3540, 261.8648,  45.0000,   4.9558, 132.3577],
        [  4.9760, 132.2280, 265.3949,  45.0000,   4.9558, 132.2320],
        [  4.9760, 132.1020, 260.7658,  45.0000,   4.9776, 132.1070],
        [  4.9760, 131.9760, 260.2252,  45.0000,   4.9777, 131.9819],
        [  4.9760, 131.8500, 259.0688,  45.0000,   4.9778, 131.8566]])


# swap columns

In [4]:
df_day_aggregated_list = []
df_day_map_list = []
for i in range(31):
    idx_for_datamap = [i*8, (i+1)*8]

    cur_map, cur_df = analysis_map_no_mm, agg_data_no_mm = data_load_instance.load_working_data_keep_ori(
    df_map, 
    idx_for_datamap, 
    ord_mm=None,  # or just omit it
    dtype=torch.float # or just omit it 
    )
    cur_df[:,0], cur_df[:,1] = cur_df[:,4] , cur_df[:,5]

    for map in cur_map.values():
        map[:,0], map[:,1] = map[:,4] , map[:,5]
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(df_day_aggregated_list[1][:10])

tensor([[  4.9764, 132.9858, 267.0307,  45.0000,   4.9764, 132.9858],
        [  4.9766, 132.8600, 258.8635,  45.0000,   4.9766, 132.8600],
        [  4.9769, 132.7346, 272.0078,  45.0000,   4.9769, 132.7346],
        [  4.9772, 132.6088, 267.8075,  45.0000,   4.9772, 132.6088],
        [  4.9776, 132.4833, 266.1405,  45.0000,   4.9776, 132.4833],
        [  4.9558, 132.3577, 261.8648,  45.0000,   4.9558, 132.3577],
        [  4.9558, 132.2320, 265.3949,  45.0000,   4.9558, 132.2320],
        [  4.9776, 132.1070, 260.7658,  45.0000,   4.9776, 132.1070],
        [  4.9777, 131.9819, 260.2252,  45.0000,   4.9777, 131.9819],
        [  4.9778, 131.8566, 259.0688,  45.0000,   4.9778, 131.8566]])


In [5]:
instance1 = kernels.vecchia_experiment(0.5, df_day_map_list[0], df_day_aggregated_list[0], nns_map, mm_cond_number, nheads=10)

a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
#a = [30.2594, 0.665, 1.8981, 0.0, 0.1317, -0.0, 1.9785]
#a = [45.1402, 0.6299, 0.7308, -0.0003, -0.0151, 0.0, 7.8922]
#a = [21.7335, 1.2817, 1.5946, 0.042, -0.1241, 0.218, 4.8654]
#a = [20.453542336448137, 1.4506118600616982, 2.43096923637867, -0.03476556019978718, -0.1559262606484541, 0.1254833595232136, 3.938183829354925]
params = torch.tensor(a, dtype=torch.float64, requires_grad=True)

cov_map = instance1.cov_structure_saver(params, instance1.matern_cov_anisotropy_v05)  
instance1.vecchia_oct22( params, instance1.matern_cov_anisotropy_v05, cov_map )


# 15994.2207 15995.6299
# 64351   64359

tensor(64359.0249, dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
lr = 0.02
lr_fast = lr_slow = lr
step = 100
gamma_par = 0.3
epochs = 500

# fit just one hour

fit on center matching

In [ ]:
# ------------------------------------
# Main Optimization Loop (Updated and Corrected)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    #idx_for_datamap = [i*8, (i+1)*8]    
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
# ... (Data loading section unchanged and correct) ...
print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]


    # Initial parameters (full vector 'a' is for reference)
    a = [28.75, 0.98, 1.5, 0, 0, 0, 1.890]
    
    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # NEW: Define learning rates and groups
    lr_slow, lr_fast = 0.02, 0.02 # Assuming these are defined elsewhere
    slow_indices = [ 1, 2, 3, 4, 5, 6] # e.g., ranges, advection, beta, nugget
    fast_indices = [0] # e.g., sigmasq
    
    # Define Parameter Groups for the optimizer
    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # Define device
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # Adjusted optimizer call: Passes the list of parameter groups
    optimizer, scheduler = model_instance.optimizer_fun(
            param_groups,     # <--- Pass the list of groups
            lr=lr,            # <--- Default LR (will be overridden by groups, but needed for function sig)
            betas=(0.9, 0.8), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # Calling the robust training loop
    out, epoch_ran = model_instance.run_vecc_scheduler_oct23(
            params_list,     # <--- Pass the list of parameter tensors
            optimizer,
            scheduler, 
            model_instance.matern_cov_anisotropy_v05, 
            epochs=epochs
        )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

Pre-loading data for all days...
Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 5000.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([28.7500], dtype=torch.float64, requires_grad=True), tensor([0.9800], dtype=torch.float64, requires_grad=True), tensor([1.0600], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([1.8900], dtype=torch.float64, requires_grad=True)]
--- Epoch 1 / Loss: 8564.823608 ---
  Param 0: Value=28.7500, Grad=-0.0993153032075611
  Param 1: Value=0.9800, Grad=11.309863882310452
  Param 2: Value=1.0600, Grad=-12.847192904026997
  Param 3: Value=0.0000, Grad=-3.339550858072471e-13
  Param 4: Value=0.0000, Grad=-5.684341886080801e-13
  Param 5: Value=0.0000, Grad=0.0
  Param 6: Value=1.8900, Grad=-3.4275484624484074
-------------

center mathicng but early stopping no scheduler

In [12]:
# ------------------------------------
# Main Optimization Loop (Corrected for List Input & Early Stopping)
# ------------------------------------
import time 
import torch
# Assuming necessary variables (lat_lon_resolution, v, mm_cond_number, 
# nheads, lr, step, gamma_par, epochs, data_load_instance, df_map, kernels, 
# nns_map) and the variables for groups (lr_slow, lr_fast, etc.) are defined.

# --- 1. Pre-load all data (unchanged) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---
days_list = [0]
# Placeholder values (assuming they are defined globally or outside the scope)
lr_slow, lr_fast = 0.02, 0.05 
slow_indices = [ 1, 2, 3, 4, 5, 6] 
fast_indices = [0] 

for day in days_list:  
    
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    # Initial parameters (full vector 'a' is for reference)
    a = [28.75, 0.98, 1.06, 0, 0, 0, 1.890]
    a = [35, 0.98, 1.06, 0, 0, 0, 1.890]
    
    # ⭐ CRITICAL CHANGE 1: Define params as a list of 1-element tensors
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # ⭐ CRITICAL CHANGE 2: Define Parameter Groups for the optimizer
    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    # Note: Cannot use params.detach().numpy() here as params is not yet concatenated
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {[p.item() for p in params_list]}')
            
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # ⭐ CRITICAL CHANGE 3: Adjusted optimizer call: Passes the list of parameter groups
    # It must now use 'param_groups' as the first argument.
    optimizer, _ = model_instance.optimizer_fun(
            param_groups,     # <--- Pass the list of groups
            lr=lr,            # <--- Default LR (overridden by groups)
            betas=(0.9, 0.8), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # ⭐ CRITICAL CHANGE 4: Early Stopping Training Call
    # New function name: run_vecc_early_stp_1028 (matching your code block)
    # Input parameter is the list of tensors: params_list
    out, epoch_ran = model_instance.run_vecc_early_stp_1028(
        params_list,     # <--- Pass the list of parameter tensors
        optimizer,
        model_instance.matern_cov_anisotropy_v05, # The covariance function
        epochs=epochs,
        patience=70,    # Example Early Stopping parameter
        min_delta=1e-3  # Example Early Stopping parameter
    )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    # The output 'out' from run_vecc_early_stp_1028 is a concatenated tensor (best_params)
    # plus the final epoch number.
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results (Concatenated Tensor):\n {out.detach().numpy()}")

print("\n--- All Days Processed ---")

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 5000.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [35.0, 0.98, 1.06, 0.0, 0.0, 0.0, 1.89]
Epoch 1/900 | NLL: 8622.638066 | Params: [35.    0.98  1.06  0.    0.    0.    1.89]
Epoch 51/900 | NLL: 8405.685392 | Params: [3.40374906e+01 1.11234065e+00 1.91866560e+00 9.32178729e-06
 2.84894614e-05 0.00000000e+00 2.49266061e+00]
Epoch 101/900 | NLL: 8277.581800 | Params: [3.49132358e+01 1.53685422e+00 2.64780792e+00 1.99547338e-05
 3.96079457e-05 0.00000000e+00 3.24166261e+00]
Epoch 151/900 | NLL: 8279.748637 | Params: [3.74382832e+01 1.58731120e+00 2.73555859e+00 2.04456758e-05
 2.47450605e-05 0.00000000e+00 3.14585079e+00]
Epoch 201/900 | NLL: 8269.555044 | Params: [4.00126311e+01 1.71158269e+00 2.94600745e+00 1.44402536e-05
 1.33050981e-05 0.00000000e+00 3.16034945e+00]
Epoch 251/900 | NLL: 8265.056434 | Params: [4.25847621e+01 1.82102251e+00 3.13294301e+00 1.933054

use swap order

In [11]:
# ------------------------------------
# Main Optimization Loop (Updated)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")

df_day_aggregated_list = []
df_day_map_list = []
for i in range(31):
    #idx_for_datamap = [i*8, (i+1)*8]
    idx_for_datamap = [i*8, i*8 +1]

    cur_map, cur_df = analysis_map_no_mm, agg_data_no_mm = data_load_instance.load_working_data_keep_ori(
    df_map, 
    idx_for_datamap, 
    ord_mm=None,  # or just omit it
    dtype=torch.float # or just omit it 
    )
    cur_df[:,0], cur_df[:,1] = cur_df[:,4] , cur_df[:,5]

    for map in cur_map.values():
        map[:,0], map[:,1] = map[:,4] , map[:,5]
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )


print(f"Data loaded for {len(df_day_map_list)} days.")
# print(df_day_aggregated_list[0].shape) # From reference code

# --- 2. Run optimization loop over pre-loaded data ---

# This list is now just for iterating
#days_list = range(len(df_day_map_list)) 
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    #a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
    a = [28.75, 0.98, 1.06, 0, 0, 0, 1.890]
    #a = [28.75, 0.98, 1.06, 0.036, -0.155, 0.179, 1.890]
    params = torch.tensor(a, dtype=torch.float64, requires_grad=True)
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params.detach().numpy()}')
            
    # --- Data loading is now done *before* the loop ---

    # We need to define the device (though we aren't passing it anymore)
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
            # device = device_str  <--- REMOVED: This was causing the TypeError
        )

    start_time = time.time()
    # Adjusted optimizer call based on expected return values (step size changed to step)
    optimizer, scheduler = model_instance.optimizer_fun(
        params, 
        lr=lr, 
        betas=(0.9, 0.8), 
        eps=1e-8, 
        step_size=step, # Using the 'step' variable here
        gamma=gamma_par  # Using gamma_par
    ) 

    # --- CRITICAL CORRECTION ---
    # 1. We no longer need to create a separate 'instance_map'.
    #    'model_instance' is already the correct instance.
    # 2. We do NOT pre-calculate 'cov_map'. The optimized training loop
    #    'run_vecc_may9' does this internally on each epoch
    #    to ensure the gradients are correct.
    # 3. We call 'run_vecc_may9' (the optimized loop) instead of 'run_vecc_grp9'.
    #    This version does not take 'cov_map' as an argument.
    
    # Calling the optimized 'run_vecc_may9'
    out, epoch_ran = model_instance.run_vecc_oct22(
        params, 
        optimizer,
        scheduler, 
        model_instance.matern_cov_anisotropy_v05, 
        epochs=epochs
    )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

print("\n--- All Days Processed ---")

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 5000.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [28.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 1, Gradients: [-1.05690956e-01  1.14465942e+01 -1.27860107e+01 -1.22070312e-04
  6.40869141e-04  0.00000000e+00 -3.60217286e+00]
 Loss: 8566.6640625, Parameters: [28.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 51, Gradients: [-1.37584448e-01  3.43799591e-01  1.42578125e-01 -7.62939453e-04
 -2.44140625e-04  0.00000000e+00 -1.18694783e+00]
 Loss: 8360.35546875, Parameters: [29.04513558  1.1734433   2.03553031  0.07186149 -0.06086177  0.
  2.9143169 ]
Epoch 101, Gradients: [-0.10074802  0.04925156  0.0145874   0.00018311 -0.00015259  0.
 -0.15212309]
 Loss: 8325.1630859375, Parameters: [ 3.00445305e+01  1.28615078e+00  2.23070790e+00 -3.06708430e-03
 -3.17577525e-02  0.00000000e+00  3.15776994e+00]
Epoch 151, Gradients: [-0.09329809 -0.00375366 -0.0032959   0.000183

## Full likelihood

Vecchia Original
[29.921775534973218, 1.8597906958307489, 3.4954059102213106, 0.22512272633105687, -0.0648045117355326, 0.0, 3.3287024509849537] 2036
(data 1250) # 1 23641

Day 1 final results: [30.882481367893934, 1.3306287573904048, 2.3063941013205755, -0.05166757348340395, -0.020577100376109176, 0.0, 3.1918623924224208] 8314. (data 5000). 


Center matching
[29.867643794098317, 1.8170445928338301, 3.452060053856289, 9.346808116428221e-06, 1.3480203304274802e-05, 0.0, 3.278045234831132, 2038.106906954074]

[30.823365978489328, 1.325000558988864, 2.300043515398019, 5.191119207004892e-06, -9.752316964224805e-07, 0.0, 3.1756405728254657, 8313.954087906111]

# 1 23644


If I fit 8 hours on original space--> numerically unstable and cholesky decomposition fails after hitting non positive matrix


## Vecchia likleihood 8 hours using same parameters ()
a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]

original   15995.6299
center matching # 15994.2207 

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [28.75   0.98   1.06   0.036 -0.155  0.179  1.89 ]
Converged at epoch 535
Epoch 536,  
 vecc Parameters: [31.00915505  1.1916293   1.84680497  0.03927127 -0.15666047  0.12771191
  4.11954968]
FINAL STATE: Epoch 536, Loss: 15746.237437383388, 
 vecc Parameters: [31.009155050569746, 1.191629302554168, 1.846804972667287, 0.03927126531202711, -0.15666046904308123, 0.12771190935579058, 4.119549677954342]
Day 1 optimization finished in 822.07s over 536 epochs.
Day 1 final results: [31.009155050569746, 1.191629302554168, 1.846804972667287, 0.03927126531202711, -0.15666046904308123, 0.12771190935579058, 4.119549677954342, 15746.237437383388]

--- All Days Processed ---

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [28.75  0.98  1.06  0.    0.    0.    1.89]
Converged at epoch 332
Epoch 333,  
 vecc Parameters: [2.98676438e+01 1.81704459e+00 3.45206005e+00 9.34680812e-06
 1.34802033e-05 0.00000000e+00 3.27804523e+00]
FINAL STATE: Epoch 333, Loss: 2038.106906954074, 
 vecc Parameters: [29.867643794098317, 1.8170445928338301, 3.452060053856289, 9.346808116428221e-06, 1.3480203304274802e-05, 0.0, 3.278045234831132]
Day 1 optimization finished in 73.78s over 333 epochs.
Day 1 final results: [29.867643794098317, 1.8170445928338301, 3.452060053856289, 9.346808116428221e-06, 1.3480203304274802e-05, 0.0, 3.278045234831132, 2038.106906954074]

--- All Days Processed ---